# CLASS: Bloom Filter

Library

In [ ]:
!pip install bitarray

import math
import time
import hashlib
import pandas as pd

from bitarray import bitarray
from google.colab import files

     |████████████████████████████████| 88 kB 6.4 MB/s 
  Created wheel for bitarray: filename=bitarray-2.3.4-cp37-cp37m-linux_x86_64.whl size=171982 sha256=4591a02483f68c76f6dc56c2e9f7f53309fbdc46f85a8c988af0e323ea8668ec
  Stored in directory: /root/.cache/pip/wheels/84/cc/5b/0e861bdb5294d22d2d4f595df936f964a95258387e11494d41
Successfully built bitarray


Class: __Bloom Filter__
> Consist:
- Function: __init__: to initialize the class, functions, and class methods functions used
- Function: __add__: to insert a specified desired element to the array
- Function: __check__: to an existence of a specified element in the array
- Function: (classmethod) __get_size__: to calculate the size of the array
- Function: (classmethod) __get_hash_count__: to calculate the hash count needed to hash a specified element

In [ ]:
class BloomFilter(object):

	'''
	Class for Bloom filter, using SHA256 hash function
	'''

	def __init__(self, items_count, fp_prob):
		'''
    Bloom Filter is a bit of array of specified size (m) and initially sets to zero

    Glosarium:
		  n = items_count : int
			  Number of items expected to be stored in bloom filter
		  p = fp_prob : float
			  False Positive probability in decimal
      k = hash count
        Hash count needed for specified value. Formula commented alongside the function.
      m = size of array
        m CAN'T BE INPUTED MANUALLY without calculating the items count and hash count. Otherwise, collision increases.
		'''

		# Initialize false positive probability in decimal
		self.fp_prob = fp_prob

		# Initialize size (m) of bit array to use
		self.size = self.get_size(items_count, fp_prob)

		# Initialize number of hash functions (k) to use
		self.hash_count = self.get_hash_count(self.size, items_count)

		# Initialize bit array of given size
    # Creating the array that will use the bloom filter method
		self.bit_array = bitarray(self.size)

		# Initialize all bits as 0
		self.bit_array.setall(0)

	@classmethod
	def from_bit_array(self, text, n, fp_prob):
		ba = bitarray(len(text))

		for i in range(len(text)):
			ba[i] = int(text[i])

		bf = BloomFilter(n, fp_prob)

		bf.bit_array = ba

		return bf

	def add(self, item):
		'''
		Encode and insert an item into the filter
		'''
		digests = []
		for i in range(self.hash_count):

			# create digest for given item.
			# using SHA256
      # checking the bit value
      # set bit value = position mod m
			digest = int(hashlib.sha256(item.encode()).hexdigest(),16) % self.size
			digests.append(digest)

			# set the bit True in bit_array
			self.bit_array[digest] = True

	def check(self, item):
		'''
		Check for existence of an item in filter
		'''
		for i in range(self.hash_count):
			digest = int(hashlib.sha256(item.encode()).hexdigest(),16) % self.size
			if self.bit_array[digest] == False:

				# if any of bit is False then,its not present
				# in filter
				# else there is probability that it exist
				return False
		return True

	@classmethod
	def get_size(self, n, p):
		'''
		Return the size of bit array(m) to used using
		following formula
		m = -(n * lg(p)) / (lg(2)^2)
		n : int
			number of items expected to be stored in filter
		p : float
			False Positive probability in decimal
		'''
		m = -(n * math.log(p))/(math.log(2)**2)
		return int(m)

	@classmethod
	def get_hash_count(self, m, n):
		'''
		Return the hash function(k) to be used using
		following formula
		k = (m/n) * lg(2)

		m : int
			size of bit array
		n : int
			number of items expected to be stored in filter
		'''
		k = (m/n) * math.log(2)
		return int(k)

# FUNCTION: create_list


In [ ]:
def create_list(array_count, element_count, fp_prob):
  '''
  Function to create multiple array that recall bloom filter method.
  '''
  array_bf = []
  a = array_count
  n = element_count
  p = fp_prob

  for i in range(a):
    BF = BloomFilter(n, p)
    array_bf.append(BF)

  return array_bf

  # baca data dari csv, insert.

# LOAD DATA CSV

In [ ]:
## Read csv dataset
source_block_df = pd.read_csv('https://gist.githubusercontent.com/alanmsmxyz/35046b1f8383cb01febaee9cca7fa565/raw/8ca9a696c0cb34102472b7247b9f17b8f303875e/block.csv')
source_index_df = pd.read_csv('https://gist.githubusercontent.com/alanmsmxyz/35046b1f8383cb01febaee9cca7fa565/raw/8ca9a696c0cb34102472b7247b9f17b8f303875e/index.csv')
keys_df = pd.read_csv('https://gist.githubusercontent.com/alanmsmxyz/35046b1f8383cb01febaee9cca7fa565/raw/8ca9a696c0cb34102472b7247b9f17b8f303875e/keys.csv')

In [ ]:
# filter n amount of block to be used
block_df = source_block_df.head(100)

# get all index for filtered blocks
latest_block = block_df['block_id'][len(block_df) - 1]
index_df = source_index_df.loc[source_index_df['block_id'] <= latest_block]

# Layer 1

## Static

In [ ]:
def create_layer_1_static(element_count):
  return create_list(element_count, 20, 0.0001)

In [ ]:
def add_layer_1_static(layer_1_static_list, block_id, block_indexes_df):
  for index in block_indexes_df['index']:
    layer_1_static_list[block_id].add(index)

In [ ]:
## Static Bloom Filter, with each bloom filter of n capacity
layer_1_static_list = create_layer_1_static(len(block_df))

timeinsert_layer1 = pd.DataFrame(data = {
    'block_id': [],
    'insert time': []
})

# insert data ke bloom filter dan hitung insert time
for block_id in block_df['block_id']:
  block_indexes_df = index_df.loc[index_df['block_id'] == block_id]

  # start timer
  t0 = time.perf_counter()

  add_layer_1_static(layer_1_static_list, block_id, block_indexes_df)

  # end timer
  t1 = time.perf_counter()
  ti = t1 - t0

  timeinsert_layer1 = timeinsert_layer1.append({
      'block_id': block_id,
      'insert time': ti,
  }, ignore_index=True)

# total storage layer 1 dalam bit
# print(layer_1_static_list[0].size * len(block_df))
# print(layer_1_static_list[0].size * len(block_df) / 8)

# for bf in layer_1_static_list:
#   size += bf.size

# print(size)

fpp_layer1 = pd.DataFrame(data = {
    'block_id': [],
    'n': [],
    'k': [],
    'fpp':[]
})

fpp = 0

# hitung fpp dan storage dari tiap BF
for i in range(len(block_df)):

  # start fpp
  n = len(index_df.loc[index_df['block_id'] == i])
  k = layer_1_static_list[i].hash_count

  fpp = pow(1 - (pow(1 - (1 / 383), (n * k))), k)

  fpp_layer1 = fpp_layer1.append({
      'block_id': i,
      'n': n,
      'k': k,
      'fpp': fpp
  }, ignore_index=True)
  # end fpp

# clean-up tipe data, set block_id ke integer di dataframe benchmark
timeinsert_layer1['block_id'] = timeinsert_layer1['block_id'].apply(int)
fpp_layer1['block_id'] = fpp_layer1['block_id'].apply(int)

In [ ]:
# export layer 1 static
layer_1_static_df = pd.DataFrame(data = {
    'block_id': [],
    'bit_array': [],
})


for i in range(len(layer_1_static_list)):
  bit_array = ''.join(str(x) for x in layer_1_static_list[i].bit_array)
  layer_1_static_df = layer_1_static_df.append({
      'block_id': i,
      'bit_array': bit_array
  }, ignore_index=True)


# set block_id sebagai integer
layer_1_static_df['block_id'] = layer_1_static_df['block_id'].apply(int)

### Query

In [ ]:
# query
timequery_layer1 = pd.DataFrame(data = {
    'block_id': [],
    'index': [],
    'query_time': [],
    'query_result': []
})

for i in index_df.index:
  block_to_query = index_df['block_id'][i]
  index_to_query = index_df['index'][i]

  t0 = time.perf_counter()

  query_result = layer_1_static_list[block_to_query].check(index_to_query)

  t1 = time.perf_counter()
  ti = t1 - t0

  timequery_layer1 = timequery_layer1.append({
      'block_id': block_to_query,
      'index': index_to_query,
      'query_time': ti,
      'query_result': query_result
  }, ignore_index=True)

# set block_id sebagai integer
timequery_layer1['block_id'] = timequery_layer1['block_id'].apply(int)

### CSVs

In [ ]:
# export layer_1 static
layer_1_static_df.to_csv('layer_1_static.csv', index=False)
files.download('layer_1_static.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# export insert time
timeinsert_layer1.to_csv('timeinsert_layer1.csv', index=False)
files.download('timeinsert_layer1.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# export fpp value
fpp_layer1.to_csv('fpp_layer1.csv', index=False)
files.download('fpp_layer1.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# export query time
timequery_layer1.to_csv('timequery_layer1.csv', index=False)
files.download('timequery_layer1.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Dynamic

In [ ]:
def create_layer_1_dynamic(fp_prob):
  new_layer_1_dynamic_list = []

  for block_id in block_df.block_id:
    n = len(index_df.loc[index_df['block_id'] == block_id])
    new_layer_1_dynamic_list.append(BloomFilter(n, fp_prob))

  return new_layer_1_dynamic_list

In [ ]:
def add_layer_1_dynamic(layer_1_dynamic_list, block_id, block_indexes_df):
    for index in block_indexes_df['index']:
      layer_1_dynamic_list[block_id].add(index)

In [ ]:
## Dynamic Bloom Filter, with n = count index for each block
layer_1_dynamic_list = create_layer_1_dynamic(0.0001)
size = 0

for block_id in block_df['block_id']:
  current_block_df = index_df.loc[index_df['block_id'] == block_id]
  add_layer_1_dynamic(layer_1_dynamic_list, block_id, current_block_df)


for bf in layer_1_dynamic_list:
  size += bf.size

print(layer_1_dynamic_list)
print(layer_1_dynamic_list[0].bit_array)
print(layer_1_dynamic_list[0].size)

print(layer_1_dynamic_list[2].bit_array)
print(layer_1_dynamic_list[2].size)

# size dynamic bloom filter dalam bit
print(size)

# fp is fixed at 0.0001

[<__main__.BloomFilter object at 0x7f5966669810>, <__main__.BloomFilter object at 0x7f5966669790>, <__main__.BloomFilter object at 0x7f595f8659d0>, <__main__.BloomFilter object at 0x7f595f865d50>, <__main__.BloomFilter object at 0x7f595f8657d0>, <__main__.BloomFilter object at 0x7f5985931510>, <__main__.BloomFilter object at 0x7f5985931dd0>, <__main__.BloomFilter object at 0x7f5985931490>, <__main__.BloomFilter object at 0x7f595f865e50>, <__main__.BloomFilter object at 0x7f595f865dd0>, <__main__.BloomFilter object at 0x7f595ed50810>, <__main__.BloomFilter object at 0x7f595ed503d0>, <__main__.BloomFilter object at 0x7f595ed502d0>, <__main__.BloomFilter object at 0x7f595ed50350>, <__main__.BloomFilter object at 0x7f595ed50850>, <__main__.BloomFilter object at 0x7f595ed508d0>, <__main__.BloomFilter object at 0x7f595ed509d0>, <__main__.BloomFilter object at 0x7f595ed50890>, <__main__.BloomFilter object at 0x7f595ed50950>, <__main__.BloomFilter object at 0x7f595ed50990>, <__main__.BloomFilt

In [ ]:
# export layer 1 dynamic
layer_1_dynamic_df = pd.DataFrame(data = {
    'block_id': [],
    'bit_array': [],
})


for i in range(len(layer_1_dynamic_list)):
  bit_array = ''.join(str(x) for x in layer_1_dynamic_list[i].bit_array)
  layer_1_dynamic_df = layer_1_dynamic_df.append({
      'block_id': i,
      'bit_array': bit_array
  }, ignore_index=True)

print(layer_1_dynamic_df)

    block_id                                          bit_array
0        0.0  0000001010000000000000000000000000000000000000...
1        1.0  0000100100001000000000000000000000000000100000...
2        2.0  0000000000000000000000000001000000000000001000...
3        3.0  0000000000000000000000000000000000000000000000...
4        4.0  0000010000000010000000000000101000000000000000...
5        5.0  0000001000000000000000000000000000000000000001...
6        6.0  0000000000000001000000000000000010000001000000...
7        7.0  0000000000000000000000000000000000010000001000...
8        8.0  0000010001000000011000000000000000000000000000...
9        9.0  0000000000000100000000000000000001000000000000...
10      10.0  0000000000001000101000000000000000000000000000...
11      11.0  0000000000000000000000000000000000000000000000...
12      12.0  0001000000000000000001000000000010000000000000...
13      13.0  0000000000000000000000000000000000000100000100...
14      14.0  00010010000000000000000000

In [ ]:
## layer 1 dynamic
layer_1_dynamic_df.to_csv('layer_1_dynamic.csv', index=False)
files.download('layer_1_dynamic.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Layer 2

In [ ]:
def get_pk_from_index(index):
  # index consist of
  # stream head hash (40)+ cipher hash (40) + pk
  return index[80:]

In [ ]:
def get_longest_pk(block_id):
  block_index_df = index_df.loc[index_df['block_id'] == block_id]

  longest_pk = ''

  for index in block_index_df['index']:
    current_index_pk = get_pk_from_index(index)

    if len(current_index_pk) > len(longest_pk):
      longest_pk = current_index_pk

  return longest_pk

In [ ]:
def pk_to_pk_code(pk):
  return ''.join(format(x, 'b') for x in bytearray(pk, 'UTF-8'))

## 2.1. UBF1

In [ ]:
# membuat ubf1 untuk 1 block
def create_ubf1(pk_code, a, n, p):
  f = len(pk_code)
 
  union_bits_count = f - a + 1

  ubf1 = create_list(union_bits_count, n, p)

  return ubf1

In [ ]:
# menambahkan data ke ubf sesuai metode pemotongan pada paper
def add_ubf1(ubf, longest_pk_code, block_id, a):
  block_index_df = index_df.loc[index_df['block_id'] == block_id]

  for index in block_index_df:
    pk = get_pk_from_index(index)
    pk_code = pk_to_pk_code(pk)

    # padd 0 (ljust) of each index pk until len(pk_code) == len(longest_pk_code)
    pk_code = pk_code.ljust(len(longest_pk_code), '0')

    for u in range (len(ubf)):
      ubf[u].add(pk_code[:a])

      pk_code = pk_code[1:]

  return(ubf)

In [ ]:
# ubf1_list[block][ubf_id]
# ubf_id : union bits count
ubf1_list = []
a = 1000
n = 20
p = 0.0001

# iterasi untuk semua blok
for block_id in block_df['block_id']:

  # cari pk terpanjang untuk setiap blok dari index
  longest_pk = get_longest_pk(int(block_id))

  # konversi pk terpanjang menjadi biner (pk_code)
  longest_pk_code = pk_to_pk_code(longest_pk)

  # create & add ubf1
  ubf1 = create_ubf1(longest_pk_code, a, n, p)
  ubf1 = add_ubf1(ubf1, longest_pk_code, block_id, a)

  ubf1_list.append(ubf1)

print(len(ubf1_list[0][0].bit_array))

383


In [ ]:
# export ubf1
# export layer 1
ubf1_df = pd.DataFrame(data = {
    'block_id': [],
    'ubf_id': [],
    'bit_array': [],
})


for i in range(len(ubf1_list)):
  for j in range(len(ubf1_list[i])):
    bit_array = ''.join(str(x) for x in ubf1_list[i][j].bit_array)

    ubf1_df = ubf1_df.append({
        'block_id': i,
        'ubf_id': j,
        'bit_array': bit_array
    }, ignore_index=True)
  
print(ubf1_df)

       block_id  ubf_id                                          bit_array
0           0.0     0.0  0000000000000000000000000000010000000000000000...
1           0.0     1.0  0000000000000000000000000000010000000000000000...
2           0.0     2.0  0000000000000000000000000000010000000000000000...
3           0.0     3.0  0000000000000000000000000000010000000000000000...
4           0.0     4.0  0000000000000000000000000000010000000000000000...
...         ...     ...                                                ...
54345      51.0  1045.0  0000000000000000000000000000010000000000000000...
54346      51.0  1046.0  0000000000000000000000000000010000000000000000...
54347      51.0  1047.0  0000000000000000000000000000010000000000000000...
54348      51.0  1048.0  0000000000000000000000000000010000000000000000...
54349      51.0  1049.0  0000000000000000000000000000010000000000000000...

[54350 rows x 3 columns]


## 2.1. UBF2

In [ ]:
# membuat ubf1 untuk 1 block
def create_ubf2(pk_code, a, n, p):
  f = len(pk_code)
 
  union_bits_count = -(f // -a) # equal to ceil(f / a)

  ubf2 = create_list(union_bits_count, n, p)

  return ubf2

In [ ]:
# menambahkan data ke ubf sesuai proposed method
def add_ubf2(ubf, longest_pk_code, block_id, a):
  block_index_df = index_df.loc[index_df['block_id'] == block_id]

  for index in block_index_df:
    pk = get_pk_from_index(index)
    pk_code = pk_to_pk_code(pk)

    # padd 0 (ljust) of each index pk until len(pk_code) == len(longest_pk_code)
    pk_code = pk_code.ljust(len(longest_pk_code), '0')

    for u in range (len(ubf)):
      ubf[u].add(pk_code[:a])

      pk_code = pk_code[a:]

  return(ubf)

In [ ]:
ubf2_list = []

# iterasi untuk semua blok
for block_id in block_df['block_id']:
  # cari pk terpanjang untuk setiap blok dari index
  longest_pk = get_longest_pk(int(block_id))

  # konversi pk terpanjang menjadi biner (pk_code)
  longest_pk_code = pk_to_pk_code(longest_pk)

  # buat UBF2 sesuai pk_code
  ubf2 = create_ubf2(longest_pk_code, 1000, 20, 0.0001)
  ubf2 = add_ubf2(ubf2, longest_pk_code, block_id, 1000)

  ubf2_list.append(ubf2)

In [ ]:
# export ubf2
# export layer 2
ubf2_df = pd.DataFrame(data = {
    'block_id': [],
    'ubf_id': [],
    'bit_array': [],
})


for i in range(len(ubf2_list)):
  for j in range(len(ubf2_list[i])):
    bit_array = ''.join(str(x) for x in ubf2_list[i][j].bit_array)

    ubf2_df = ubf2_df.append({
        'block_id': i,
        'ubf_id': j,
        'bit_array': bit_array
    }, ignore_index=True)
  
print(ubf2_df)

     block_id  ubf_id                                          bit_array
0         0.0     0.0  0000000000000000000000000000010000000000000000...
1         0.0     1.0  0000000000000000000000000000010000000000000000...
2         0.0     2.0  0000000000000000000000000000000000000000000000...
3         1.0     0.0  0000000000000000000000000000010000000000000000...
4         1.0     1.0  0000000000000000000000000000010000000000000000...
..        ...     ...                                                ...
151      50.0     1.0  0000000000000000000000000000010000000000000000...
152      50.0     2.0  0000000000000000000000000000000000000000000000...
153      51.0     0.0  0000000000000000000000000000010000000000000000...
154      51.0     1.0  0000000000000000000000000000010000000000000000...
155      51.0     2.0  0000000000000000000000000000000000000000000000...

[156 rows x 3 columns]


## 2.2 DBF

In [ ]:
def create_dbf(pk_code):
  return create_list(len(pk_code), 20, 0.001 )

In [ ]:
def add_dbf(dbf, pk_code, block_id):
  for i in range(len(pk_code)):
    if int(pk_code[i]) == 0: continue

    block_index_df = index_df.loc[index_df['block_id'] == block_id]

    for index in block_index_df:
      dbf[i].add(index)

  return dbf

In [ ]:
# dfb_list[block_id][pk_code_pos]
dbf_list = []

for block_id in block_df['block_id']:
  # cari pk terpanjang untuk setiap blok dari index
  longest_pk = get_longest_pk(int(block_id))

  # konversi pk terpanjang menjadi biner (pk_code)
  pk_code = pk_to_pk_code(longest_pk)

  # buat DBF sesuai pk_code
  dbf = create_dbf(pk_code)
  dbf = add_dbf(dbf, pk_code, block_id)

  dbf_list.append(dbf)

In [ ]:
print(dbf_list[0])
print(dbf_list[0][0].bit_array)

[<__main__.BloomFilter object at 0x7f595c2cf650>, <__main__.BloomFilter object at 0x7f595c2cf610>, <__main__.BloomFilter object at 0x7f595c2cf750>, <__main__.BloomFilter object at 0x7f595c2cf690>, <__main__.BloomFilter object at 0x7f595c2cf6d0>, <__main__.BloomFilter object at 0x7f595c2cf7d0>, <__main__.BloomFilter object at 0x7f595c2cf850>, <__main__.BloomFilter object at 0x7f595c2cf890>, <__main__.BloomFilter object at 0x7f595c2cf810>, <__main__.BloomFilter object at 0x7f595c2cf790>, <__main__.BloomFilter object at 0x7f595c2cf8d0>, <__main__.BloomFilter object at 0x7f595c2cf910>, <__main__.BloomFilter object at 0x7f595c2cf950>, <__main__.BloomFilter object at 0x7f595c2cf990>, <__main__.BloomFilter object at 0x7f595c2cf9d0>, <__main__.BloomFilter object at 0x7f595c2cfa10>, <__main__.BloomFilter object at 0x7f595c2cfa50>, <__main__.BloomFilter object at 0x7f595c2cfa90>, <__main__.BloomFilter object at 0x7f595c2cfad0>, <__main__.BloomFilter object at 0x7f595c2cfb10>, <__main__.BloomFilt

In [ ]:
# export dbf
# export layer 1 static
dbf_df = pd.DataFrame(data = {
    'block_id': [],
    'ubf_id': [],
    'bit_array': [],
})


for i in range(len(dbf_list)):
  for j in range(len(dbf_list[i])):
    bit_array = ''.join(str(x) for x in dbf_list[i][j].bit_array)

    dbf_df = dbf_df.append({
        'block_id': i,
        'ubf_id': j,
        'bit_array': bit_array
    }, ignore_index=True)
  
print(dbf_df)

        block_id  ubf_id                                          bit_array
0            0.0     0.0  0000000000000000000000000000000000000000000000...
1            0.0     1.0  0000000000000000000000000000000000000000000000...
2            0.0     2.0  0000000000000000000000000000000000000000000000...
3            0.0     3.0  0000000000000000000000000000000000000000000000...
4            0.0     4.0  0000000000000000000000000000000000000000000000...
...          ...     ...                                                ...
106293      51.0  2044.0  0000000000000000000000000000000000000000000000...
106294      51.0  2045.0  0000000000000000000000000000000000000000000000...
106295      51.0  2046.0  0000000000000000000000000000000000000000000000...
106296      51.0  2047.0  0000000000000000000000000000000000000000000000...
106297      51.0  2048.0  0000000000000000000000000000000000000000000000...

[106298 rows x 3 columns]


Download CSV Layer 2

In [ ]:
## UBF1
ubf1_df.to_csv('ubf1.csv', index=False)
files.download('ubf1.csv')

## UBF2
ubf2_df.to_csv('ubf2.csv', index=False)
files.download('ubf2.csv')

## DBF
dbf_df.to_csv('dbf.csv', index=False)
files.download('dbf.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>